#### Importacion de librerias

In [ ]:
import pandas as pd
import numpy as np

#### Ingesta de los datos

In [ ]:
archivo = ('Data/reto_agua.csv')
df = pd.read_csv(archivo)


#### Analisis Exploratorio y Transformacion de los Datos

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()


In [ ]:
df.describe()


In [ ]:
df.funder.value_counts().sum()            

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(columns='id')

In [ ]:
df.duplicated().sum()

In [ ]:
df.wpt_name.value_counts()

In [ ]:
df.info()

In [ ]:
df = df.replace({'none': np.nan, None: np.nan})

In [ ]:
df = df.replace({'None': np.nan, None: np.nan})

In [ ]:
min = ['funder', 'installer', 'wpt_name', 'basin', 'region', 'public_meeting', 'recorded_by',
'scheme_management', 'permit', 'extraction_type', 'management_group', 'payment_type', 'water_quality',
'quality_group', 'quantity_group', 'source_class', 'waterpoint_type_group', 'status_group']

def minusculas(df, columnas):
    for i in columnas:
        df[i] = df[i].apply(lambda x: x.lower() if isinstance(x, str) else x)
    return df

print("Valores NaN antes de aplicar la función:")
print(df.isna().sum())
df = minusculas(df, min)


In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

#### Analisis de columnas en forma individual

In [ ]:
def analyze_column(df, column_name):
  
    col_data = df[column_name]
    result = {}
    result['column_name'] = column_name
    result['dtype'] = col_data.dtype
    result['missing_values'] = col_data.isnull().sum()
    result['missing_percentage'] = (col_data.isnull().sum() / len(col_data)) * 100
    
    if pd.api.types.is_numeric_dtype(col_data):
        result['description'] = col_data.describe()
    elif pd.api.types.is_bool_dtype(col_data):
        result['value_counts'] = col_data.value_counts()
    elif pd.api.types.is_string_dtype(col_data):
        result['description'] = col_data.describe()
        result['value_counts'] = col_data.value_counts()
    else:
        result['description'] = col_data.describe(include='all')
        result['value_counts'] = col_data.value_counts()
    
    return result


def print_analysis(analysis):
    
    print(f"\nAnálisis para la columna: {analysis['column_name']}")
    print(f"Tipo de dato: {analysis['dtype']}")
    print(f"Valores nulos: {analysis['missing_values']} ({analysis['missing_percentage']:.2f}%)")
    
    if 'description' in analysis:
        print("\nEstadísticas descriptivas:")
        print(analysis['description'])
    
    if 'outliers' in analysis:
        print(f"\nValores atípicos (outliers): {analysis['outliers']}")
        if analysis['outliers'] > 0:
            print(analysis['outliers_data'])
    
    if 'value_counts' in analysis:
        print("\nFrecuencia de valores:")
        print(analysis['value_counts'])

Mauro

In [ ]:
columns_to_analyze = ['population','public_meeting','recorded_by', 'scheme_management', 'permit']

In [ ]:
# Bucle para analizar cada columna en la lista y almacenar resultados
analyses = []
for column in columns_to_analyze:
    analysis = analyze_column(df, column)
    analyses.append(analysis)

for analysis in analyses:
    print_analysis(analysis)

In [ ]:
#### Columna 'population'
# Se Mantiene la columna sin cambios.
# Es importante entender por qué hay tantos valores de población cero.
# Podría ser útil investigar si estos pozos están en áreas completamente deshabitadas 
# o si se trata de áreas rurales muy dispersas.

In [ ]:
#### Columna 'public_meeting' Faltan 5.77%
# Reemplazar los valores faltantes con True pq True es top frecuencia con 47292 y categorizar
df['public_meeting'] = df['public_meeting'].fillna(True)
df['public_meeting'] = df['public_meeting'].astype(int)
df['public_meeting'] = df['public_meeting'].astype('int8')

In [ ]:
#### Columna 'recorded_by' 1 registro cargo todos los datoa es irrelevante drop
# Eliminar la columna 'recorded_by'
df = df.drop(columns='recorded_by')

In [ ]:
#### Columna 'scheme_management' faltan 6.64% de los datos 
#Imputar valores faltantes con la categoría más frecuente (VWC) 34459 y luego categorizar
df['scheme_management'] = df['scheme_management'].fillna('VWC')
categories = df['scheme_management'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['scheme_management'] = df['scheme_management'].apply(lambda x: category_map[x])
df['scheme_management'] = df['scheme_management'].astype('int64')

In [ ]:
#### Columna 'permit' 
# Reemplazar los valores faltantes (5%) con True pq True es top frecuencia con 47292
df['permit'] = df['permit'].fillna(True) 
df['permit'] = df['permit'].astype(int) # Convertir a binario: True -> 1, False -> 0
df['permit'] = df['permit'].astype('int8')


Ariel

In [ ]:
columns_to_analyze = ['quality_group', 'quantity_group', 'source_class', 'waterpoint_type_group','status_group']

In [ ]:
analyses = []
for column in columns_to_analyze:
    analysis = analyze_column(df, column)
    analyses.append(analysis)

for analysis in analyses:
    print_analysis(analysis)

In [ ]:
#### Columna 'quality_group'
# Renombrar la columna 'quality_group' a 'water_quality_2'y luego categorizar
df.rename(columns={'quality_group': 'water_quality_2'}, inplace=True)
categories = df['water_quality_2'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['water_quality_2'] = df['water_quality_2'].apply(lambda x: category_map[x])
df['water_quality_2'] = df['water_quality_2'].astype('int64')

In [ ]:
#### Columna 'quantity_group' se categoriza
categories = df['quantity_group'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['quantity_group'] = df['quantity_group'].apply(lambda x: category_map[x])
df['quantity_group'] = df['quantity_group'].astype('int64')

In [ ]:
#### Columna 'source_class'
categories = df['source_class'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['source_class'] = df['source_class'].apply(lambda x: category_map[x])
df['source_class'] = df['source_class'].astype('int64')

In [ ]:
#### Columna 'waterpoint_type_group'
categories = df['waterpoint_type_group'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['waterpoint_type_group'] = df['waterpoint_type_group'].apply(lambda x: category_map[x])
df['waterpoint_type_group'] = df['waterpoint_type_group'].astype('int64')

In [ ]:
#### Columna 'status_group'
categories = df['status_group'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['status_group'] = df['status_group'].apply(lambda x: category_map[x])
df['status_group'] = df['status_group'].astype('int64')

In [ ]:
df_10colum = df.copy()

Ariel: 

In [ ]:
columns_to_analyze = ['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude']

In [ ]:
analyses = []
for column in columns_to_analyze:
    analysis = analyze_column(df, column)
    analyses.append(analysis)

for analysis in analyses:
    print_analysis(analysis)

In [ ]:
df['amount_tsh'] = df['amount_tsh'].astype('int64')

In [ ]:
df = df.dropna(subset=['funder'])

funder_map = {
    'government of tanzania': 1,
    'danida': 2,
    'hesawa': 3,
    'rwssp': 4,
    'world bank': 5
}
df['funder'] = df['funder'].apply(lambda x: funder_map.get(x, 6))

In [ ]:
#gps_height no la toco, esta ok. reviso -90 y no es un outliers
df.gps_height.sort_values(ascending=False)

In [ ]:
df.installer.value_counts()

In [ ]:
#installer tiene Valores nulos: 3215 (5.84%)
df = df.dropna(subset=['installer'])

funder_map = {
    'dwe': 1,
    'government': 2,
    'hesawa': 3,
    'rwe': 4,
    'commu': 5
}
df['installer'] = df['installer'].apply(lambda x: funder_map.get(x, 6))

In [ ]:
# Filtrar los registros anómalos de coordenadas en total
anomalous_records = (df['longitude'] == 0) & (np.abs(df['latitude']) < 1e-6)
df = df[~anomalous_records]

Erick

In [ ]:
columns_to_analyze = ['latitude','wpt_name', 'num_private', 'basin', 'region']


In [ ]:
analyses = []
for column in columns_to_analyze:
    analysis = analyze_column(df, column)
    analyses.append(analysis)

for analysis in analyses:
    print_analysis(analysis)

In [ ]:
df.wpt_name.value_counts()

In [ ]:
#wpt_name
df = df.dropna(subset=['wpt_name'])

funder_map = {
    'shuleni': 1,
    'zahanati': 2,
    'msikitini': 3,
    'kanisani': 4,
    'ofisini': 5
}
df['wpt_name'] = df['wpt_name'].apply(lambda x: funder_map.get(x, 6))

In [ ]:
df.wpt_name.value_counts()

In [ ]:
df = df.drop(columns='num_private')

In [ ]:
#region categorizar
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['region'] = label_encoder.fit_transform(df['region'])

In [ ]:
#BASIN categorizar
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['basin'] = label_encoder.fit_transform(df['basin'])

Gabriel

In [ ]:
#Modificación columnas object a int64

df['payment_type'] = df['payment_type'].apply(lambda x: 0 if x == 'unknown' 
                                                else (1 if x == 'other' else (2 if x == 'on failure' 
                                                    else (3 if x == 'never pay' else(4 if x == 'annually' else(5 if x == 'monthly' else 6))))))
df.payment_type.unique()

df['management_group'] = df['management_group'].apply(lambda x: 0 if x == 'unknown' 
                                                else (1 if x == 'other' else (2 if x == 'parastatal' 
                                                    else (3 if x == 'commercial' else 4))))
df.management_group.unique()

In [ ]:
df = df.drop(columns='water_quality')

In [ ]:
df.construction_year.value_counts()
df.construction_year.unique()

In [ ]:
# Calcular la mediana del año de construcción para pozos funcionales y no funcionales
median_functional = df[df['status_group'] == 1]['construction_year'].replace(0, pd.NA).median()
median_non_functional = df[df['status_group'] == 0]['construction_year'].replace(0, pd.NA).median()

def fill_missing_year(row):
    if row['construction_year'] == 0:
        if row['status_group'] == 1:
            return median_functional
        else:
            return median_non_functional
    return row['construction_year']

df['construction_year'] = df.apply(fill_missing_year, axis=1)
df['construction_year'] = df['construction_year'].astype(int)

In [ ]:
#Extraction_type categorizar
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['extraction_type'] = label_encoder.fit_transform(df['extraction_type'])

#### Creacion de *CSV* con la Data preparada

In [ ]:
df.to_csv('Data/Reto_agua_final.csv', index=False)

#### Correlacion

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el archivo CSV
file_path = 'Reto_agua_final.csv'
df = pd.read_csv(file_path)

# Calcular la matriz de correlación
correlation_matrix = df.corr()

# Configurar el tamaño de la figura
plt.figure(figsize=(12, 10))

# Crear el mapa de calor
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)

# Configurar los títulos
plt.title('Matriz de Correlación', size=15)
plt.show()


#### 2.2. Ejercicio 2 

Ahora, vamos a entrenar el modelo: 

• Dividid los datos en variable independiente y target 
• Dividid el modelo en un conjunto de datos para el test (20%) y otro para el train (80%) 
y random_state=42 
• Entrenad varios modelos con los datos de train, validadlo con el test y seleccionad 
el que mejor resultado obtiene. 
Una vez hecho esto, responded a las siguientes preguntas: 



• ¿Qué score da el de entrenamiento y con el test? 

• ¿Creéis que puede tener sobreajuste (overfitting) o infraajuste (underfitting)?

Respuesta posible luego de evaluar los modelo:

Logistic Regression
Observación: Las precisiones en el conjunto de entrenamiento y prueba son iguales (0.75).
Conclusión: Este modelo no muestra signos de sobreajuste ni infraajuste, pero su desempeño es relativamente bajo, lo que sugiere que podría no ser el mejor modelo para estos datos.

K-Nearest Neighbors (KNN)
Observación: La precisión en el conjunto de entrenamiento (0.87) es mayor que en el conjunto de prueba (0.80).
Conclusión: Hay una ligera diferencia, lo que indica un posible sobreajuste, aunque no es extremo. El modelo generaliza relativamente bien.

Random Forest
Observación: La precisión en el conjunto de entrenamiento es perfecta (1.00) mientras que en el conjunto de prueba es alta (0.91).
Conclusión: Este modelo muestra claros signos de sobreajuste, ya que tiene una precisión perfecta en el conjunto de entrenamiento. Sin embargo, aún generaliza bien en el conjunto de prueba.

XGBoost
Observación: La precisión en el conjunto de entrenamiento (0.93) es ligeramente mayor que en el conjunto de prueba (0.90).
Conclusión: Hay un pequeño margen de diferencia, lo que sugiere un buen equilibrio entre ajuste y generalización. Este modelo no muestra signos significativos de sobreajuste ni infraajuste.

Resumen

Logistic Regression: No muestra signos de sobreajuste ni infraajuste, pero tiene un desempeño moderado.

K-Nearest Neighbors: Ligero sobreajuste, pero generaliza bien.

Random Forest: Muestra signos claros de sobreajuste.

XGBoost: Buen equilibrio entre ajuste y generalización, siendo el mejor modelo en términos de precisión y ROC-AUC.

-------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------

#### Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split

# Suponiendo que la columna 'status_group' es la etiqueta a predecir
X = df.drop('status_group', axis=1)
y = df['status_group']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score


# Cargar el archivo CSV
file_path = 'Reto_agua_final.csv'
df = pd.read_csv(file_path)


# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'Logistic Regression': LogisticRegression(max_iter=2000),  # Aumentar el número de iteraciones
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(eval_metric='mlogloss')
}


results = {}

# Entrenar y evaluar cada modelo
for name, model in models.items():
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    results[name] = {
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy
    }

# Mostrar los resultados
results


Elección del Mejor Modelo
Basado en las métricas proporcionadas, el modelo de XGBoost muestra un rendimiento excelente con una buena precisión, recall y ROC-AUC, y no presenta un sobreajuste tan marcado como el Random Forest.

Aca se evalua el df completo

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt


# Cargar el archivo CSV
file_path = 'Reto_agua_final.csv'
df = pd.read_csv(file_path)

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    if model_name == 'Logistic Regression':
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    
    print(f"Evaluating {model_name}")
    print("Training Classification Report:")
    print(classification_report(y_train, y_pred_train))
    
    print("Testing Classification Report:")
    print(classification_report(y_test, y_pred_test))
    
    
    if hasattr(model, "predict_proba"):
        y_probs = model.predict_proba(X_test)[:, 1]
    else: 
        y_probs = model.decision_function(X_test)
    
    fpr, tpr, _ = roc_curve(y_test, y_probs, pos_label=1)
    roc_auc = auc(fpr, tpr)
    print(f"ROC-AUC Score: {roc_auc}")
    
for name, model in models.items():
    evaluate_model(model, X_train, y_train, X_test, y_test, name)


#### 2.3. Ejercicio 3 

Seleccionad las 21 variables que más influyen en la predicción y entrenad de nuevo el 
modelo. ¿Mejora? 

Usadlas para sacar los scoring ['accuracy', 'precision', 'recall'] del conjunto de train: 
• ¿Interpreta accuracy? 
• ¿Interpreta precision? 
• ¿Interpreta recall? 
• ¿Predice mejor los positivos o los negativos? 

Sacamos las 21 las 21 variables que más influyen en la predicción

In [ ]:
import pandas as pd

file_path = 'Reto_agua_final.csv'
df = pd.read_csv(file_path)

# Calcular la correlación de cada característica con la variable objetivo 'status_group'
correlation_matrix = df.corr()
correlation_with_target = correlation_matrix['status_group'].abs()

# Seleccionar las 21 características con la mayor correlación absoluta con 'status_group'
top_21_features = correlation_with_target.sort_values(ascending=False).head(22).index.tolist()
top_21_features.remove('status_group')  # Remover 'status_group' de la lista

print("Las 21 variables que mejor correlacionan con 'status_group' son:")
print(top_21_features)


Volvemos a correr los modelos pero ahora con las 21 variables seleccionadas

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt


selected_columns = ['construction_year', 'quantity_group', 'payment_type', 'waterpoint_type_group', 'funder', 
                    'gps_height', 'scheme_management', 'extraction_type', 'water_quality_2', 'public_meeting', 
                    'region', 'permit', 'latitude', 'source_class', 'amount_tsh', 'basin', 'population', 'wpt_name', 
                    'management_group', 'installer', 'longitude']

# Filtrar el DataFrame original para incluir solo las columnas seleccionadas
X = df[selected_columns]
y = df['status_group']

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    if model_name == 'Logistic Regression':
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    
    print(f"Evaluating {model_name}")
    print("Training Classification Report:")
    print(classification_report(y_train, y_pred_train))
    
    print("Testing Classification Report:")
    print(classification_report(y_test, y_pred_test))
    
    
    if hasattr(model, "predict_proba"):
        y_probs = model.predict_proba(X_test)[:, 1]
    else: 
        y_probs = model.decision_function(X_test)
    
    fpr, tpr, _ = roc_curve(y_test, y_probs, pos_label=1)
    roc_auc = auc(fpr, tpr)
    print(f"ROC-AUC Score: {roc_auc}")
    
for name, model in models.items():
    evaluate_model(model, X_train, y_train, X_test, y_test, name)

En base a ['accuracy', 'precision', 'recall']  

• ¿Interpreta accuracy? 
• ¿Interpreta precision? 
• ¿Interpreta recall? 
• ¿Predice mejor los positivos o los negativos?

Interpreta accuracy:

Logistic Regression: Accuracy es moderada (0.75), indicando que el modelo predice correctamente el 75% de las veces.

K-Nearest Neighbors: Accuracy es alta en el entrenamiento (0.87) pero baja en la prueba (0.80), sugiriendo posible sobreajuste.

Random Forest: Accuracy es perfecta en el entrenamiento (1.00) y alta en la prueba (0.91), indicando sobreajuste.

XGBoost: Accuracy es alta en ambos conjuntos (0.93 en entrenamiento y 0.90 en prueba), mostrando buen equilibrio y generalización.

Interpreta Precision:


Logistic Regression: Precision es moderada, sugiriendo que una proporción significativa de las predicciones positivas no son correctas.

K-Nearest Neighbors: Precision es alta, especialmente para la clase 0 en el conjunto de entrenamiento.

Random Forest: Precision es perfecta en el entrenamiento, indicando sobreajuste, pero alta en la prueba.

XGBoost: Precision es alta y equilibrada en ambas clases y en ambos conjuntos, indicando buen rendimiento.

Interpreta Recall:

Logistic Regression: Recall es moderada, especialmente baja para la clase 1.

K-Nearest Neighbors: Recall es alta en el conjunto de entrenamiento, pero disminuye en el conjunto de prueba, sugiriendo sobreajuste.

Random Forest: Recall es perfecta en el entrenamiento, indicando sobreajuste, pero alta en la prueba.

XGBoost: Recall es alta y equilibrada en ambas clases y en ambos conjuntos, indicando buen rendimiento.

¿Predice mejor los positivos o los negativos?:

Logistic Regression: Predice mejor los negativos (clase 0), dado que tanto la precisión como el recall son más altos para la clase 0 en comparación con la clase 1.

K-Nearest Neighbors: Predice mejor los negativos (clase 0) tanto en el entrenamiento como en la prueba, aunque hay una disminución en el rendimiento para ambas clases en el conjunto de prueba.

Random Forest: Predice bien ambas clases en el conjunto de prueba, aunque hay una ligera ventaja para los negativos (clase 0).

XGBoost: Predice bien ambas clases, con una ligera ventaja para los negativos (clase 0) en términos de recall en el conjunto de prueba.

Comparación con Resultados Anteriores

Logistic Regression: No hay mejora significativa en los resultados.

K-Nearest Neighbors: Similar rendimiento, con ligero sobreajuste.

Random Forest: Similar rendimiento, sigue mostrando sobreajuste.

XGBoost: Muestra una ligera mejora en la generalización y sigue siendo el mejor modelo en términos de equilibrio entre precisión y recall.

Conclusión

XGBoost sigue siendo el mejor modelo en términos de rendimiento general y balance entre las métricas.

Random Forest muestra sobreajuste significativo.

K-Nearest Neighbors muestra un ligero sobreajuste.

Logistic Regression tiene un rendimiento moderado y consistente, pero no es el mejor modelo para este problema.

El uso de las 21 variables más importantes parece mantener o mejorar ligeramente el rendimiento de XGBoost, indicando que estas variables son relevantes y suficientes para una buena predicción.


#### 2.4. Ejercicio 4 

Validad la correlación con uno o más gráficos con las columnas ['amount_tsh', 'funder', 
'gps_height', 'installer', 'longitude', 'latitude', 'num_private', 'basin','status_group']. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo CSV
file_path = 'Reto_agua_final.csv'
data = pd.read_csv(file_path)


# Gráfico de dispersión para 'amount_tsh' vs 'gps_height'
plt.figure(figsize=(10, 6))
plt.scatter(data['amount_tsh'], data['gps_height'], alpha=0.5)
plt.title('Scatter Plot: amount_tsh vs gps_height')
plt.xlabel('amount_tsh')
plt.ylabel('gps_height')
plt.grid(True)
plt.show()

# Gráfico de dispersión para 'longitude' vs 'latitude'
plt.figure(figsize=(10, 6))
plt.scatter(data['longitude'], data['latitude'], alpha=0.5)
plt.title('Scatter Plot: longitude vs latitude')
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.grid(True)
plt.show()

# Diagrama de cajas para 'amount_tsh' vs 'status_group'
plt.figure(figsize=(10, 6))
sns.boxplot(x='status_group', y='amount_tsh', data=data)
plt.title('Box Plot: amount_tsh vs status_group')
plt.xlabel('status_group')
plt.ylabel('amount_tsh')
plt.grid(True)
plt.show()

# Diagrama de cajas para 'gps_height' vs 'status_group'
plt.figure(figsize=(10, 6))
sns.boxplot(x='status_group', y='gps_height', data=data)
plt.title('Box Plot: gps_height vs status_group')
plt.xlabel('status_group')
plt.ylabel('gps_height')
plt.grid(True)
plt.show()

# Diagrama de cajas para 'population' vs 'status_group'
plt.figure(figsize=(10, 6))
sns.boxplot(x='status_group', y='population', data=data)
plt.title('Box Plot: population vs status_group')
plt.xlabel('status_group')
plt.ylabel('population')
plt.grid(True)
plt.show()

# Mapa de calor para las correlaciones entre variables numéricas
numerical_columns = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population']
correlation_matrix = data[numerical_columns].corr()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Heatmap: Correlation Matrix')
plt.show()



Después, haced un gráfico, el que consideréis adecuado, para detectar outliers en 
population y gps_height ¿alguno tiene outliers?


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = 'Reto_agua_final.csv'
data = pd.read_csv(file_path)

# Diagrama de cajas para 'population'
plt.figure(figsize=(10, 6))
sns.boxplot(x=data['population'])
plt.title('Box Plot: population')
plt.xlabel('population')
plt.grid(True)
plt.show()

# Diagrama de cajas para 'gps_height'
plt.figure(figsize=(10, 6))
sns.boxplot(x=data['gps_height'])
plt.title('Box Plot: gps_height')
plt.xlabel('gps_height')
plt.grid(True)
plt.show()


Sí, el gráfico de caja para la columna population muestra claramente la presencia de outliers.

De ser así, eliminadlos con el método de 
Inter cuartil con la columna o columnas con datos atípicos.


In [ ]:
df.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo CSV
file_path = 'Reto_agua_final.csv'
data = pd.read_csv(file_path)

# Definir una función para eliminar outliers usando el método IQR para una columna específica
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Eliminar outliers en la columna 'population'
df = remove_outliers_iqr(data, 'population')

# Diagrama de cajas para 'population' después de eliminar outliers
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['population'])
plt.title('Box Plot: population (sin outliers)')
plt.xlabel('population')
plt.grid(True)
plt.show()


In [ ]:
df.shape

El gráfico generado muestra que aún existen algunos valores considerados como outliers en la columna population, sin embargo son valores muy proximos al limite superior y estan concentrados entre los 500 y 650 pobladores.


 ¿El modelo ha mejorado? 
Recordad que hay que volver a sacar los valores x e y (test y train). 

In [ ]:
df.shape

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

X = data.drop(columns=['status_group'])
y = data['status_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modelo XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

# prueba
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')
test_classification_rep = classification_report(y_test, y_test_pred)

# entrenamiento
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_classification_rep = classification_report(y_train, y_train_pred)

print("Testing Metrics:")
print(f'Accuracy: {test_accuracy}')
print(f'Precision: {test_precision}')
print(f'Recall: {test_recall}')
print('Classification Report:')
print(test_classification_rep)

print("\nTraining Metrics:")
print(f'Accuracy: {train_accuracy}')
print(f'Precision: {train_precision}')
print(f'Recall: {train_recall}')
print('Classification Report:')
print(train_classification_rep)


####  ¿El modelo ha mejorado?  

*Entrenamiento*: Las métricas de entrenamiento son muy similares entre las nuevas y las anteriores. No hay una mejora significativa.

*Prueba:* Las métricas de prueba también son muy similares entre las nuevas y las anteriores.
En general, no hay una mejora significativa entre las nuevas métricas y las anteriores. Ambas versiones del modelo muestran un rendimiento similar tanto en los conjuntos de entrenamiento como en los de prueba. Sin embargo, el nuevo modelo muestra un rendimiento ligeramente inferior en términos de precision y recall en el conjunto de prueba.

--------------------------------------------------------------------------------------

Afinamos el modelo usando GridSearchCV

Afinando el modelo

El modelo XGBoost afinado muestra un excelente rendimiento tanto en el conjunto de entrenamiento como en el de prueba.
El modelo no parece tener problemas significativos de sobreajuste o infraajuste.
El alto ROC-AUC Score (0.9645) indica una excelente capacidad de discriminación entre las clases.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_model = xgb.XGBClassifier(eval_metric='mlogloss')
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_xgb_model = grid_search.best_estimator_
evaluate_model(best_xgb_model, X_train, y_train, X_test, y_test, "XGBoost (Tuned)")


#### Conclusión sobre el ajuste de Hyperparametros

Entrenamiento:

Las métricas de entrenamiento han mejorado significativamente con el ajuste de los hiperparámetros. La precisión, el recall y el f1-score han aumentado a 0.96, lo que indica que el modelo está ajustándose mejor a los datos de entrenamiento.

Prueba:

Las métricas de prueba se mantienen aproximadamente igual, con una precisión, recall y f1-score de 0.90. Sin embargo, el ROC-AUC Score ha mejorado ligeramente de 0.9627 a 0.9645.
El ajuste de hiperparámetros ha mejorado el rendimiento del modelo en los datos de entrenamiento, lo que sugiere que el modelo está mejor ajustado. Aunque las métricas en el conjunto de prueba no han mejorado significativamente, la ligera mejora en el ROC-AUC Score indica un mejor rendimiento general del modelo ajustado.